In [1]:
import h5py
import numpy as np
from transformers import AutoTokenizer
import wandb
import matplotlib.pyplot as plt
np.random.seed(42)


/opt/anaconda3/envs/qwen/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Baseline


## (a) Preprocesses and tokenized the dataset

In [2]:
from src.preprocessor import load_and_preprocess

### Use the preprocesser to preprocess the dataset, and tokenize them

In [3]:
file_path = "lotka_volterra_data.h5"

# Use the function to load and preprocess the data
train_texts, val_texts, test_texts = load_and_preprocess(
    file_path,
    decimal_places=2,
    max_target_value=9.99
)

# Demonstrate tokenization using Qwen2.5
model_name = "Qwen/Qwen2.5-0.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

tokenized_train=[]
tokenized_val=[]
tokenized_test=[]
for i in range(len(train_texts)):
    tokenized_train.append(tokenizer(train_texts[i], return_tensors="pt", add_special_tokens=False)["input_ids"][0])
for i in range(len(val_texts)): 
    tokenized_val.append(tokenizer(val_texts[i], return_tensors="pt", add_special_tokens=False)["input_ids"][0])    
for i in range(len(test_texts)):
    tokenized_test.append(tokenizer(test_texts[i], return_tensors="pt", add_special_tokens=False)["input_ids"][0])





### Show examples of preprocessed data and tokenized data

In [4]:
print(f"Example Preprocessed Sequences: Train Sequence {1}:", train_texts[1][:40])
print(f"shape of train text:", len(train_texts[1]))
print(f"Example Tokenized Sequence: Train Sequence {1}:",tokenized_train[1].tolist()[:40])
print(f"shape of tokenized train text", len(tokenized_train[1]))

Example Preprocessed Sequences: Train Sequence 1: 0.92,0.74;0.56,0.77;0.34,0.70;0.23,0.59;
shape of train text: 999
Example Tokenized Sequence: Train Sequence 1: [15, 13, 24, 17, 11, 15, 13, 22, 19, 26, 15, 13, 20, 21, 11, 15, 13, 22, 22, 26, 15, 13, 18, 19, 11, 15, 13, 22, 15, 26, 15, 13, 17, 18, 11, 15, 13, 20, 24, 26]
shape of tokenized train text 999


## (b) Evaluate the untrained Qwen2.5-Instruct model’s forecasting ability on this tokenized dataset.

In [5]:
from src.qwen import load_qwen

In [6]:
import torch
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
original_model, tokenizer = load_qwen()
original_model.to(device)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((896,), eps=1e-06)
    (rotary_emb): Qwen2RotaryEmbe

In [7]:
from src.evaluation import *



In [8]:

wandb.init(project="Qwen_baseline", name="untrained-noLora_evaluation",reinit=True) 
avg_loss, avg_mse = evaluation(
    original_model, tokenizer, tokenized_val, context_ratio=0.7
)


print("=== Baseline Evaluation on Untrained Qwen2.5-Instruct ===")
print(f"Average Cross-Entropy Loss: {avg_loss:.4f}")
print(f"Average MSE (Forecast):  {avg_mse:.4f}")



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possibl

/opt/anaconda3/envs/qwen/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/anaconda3/envs/qwen/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:634: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/opt/anaconda3/envs/qwen/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:651: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
The attention mask is not set and cannot be inferred from i

=== Baseline Evaluation on Untrained Qwen2.5-Instruct ===
Average Cross-Entropy Loss: 1.7850
Average MSE (Forecast):  0.2702


## (c) Map each operation to its flops

In [37]:
from src.flops import flops_for_experiment
import numpy as np

# Suppose we want to run inference for 10 steps (e.g., 10 forward passes)
num_steps = 500

# Hypothetical model configuration
batch_size = 4    # number of samples per batch
seq_len = 512  # input sequence length
hidden_dim = 896   # model hidden dimension
num_layers = 24    # number of Transformer blocks
num_heads = 14     # number of attention heads
r=4                 # Lora rank
ffn_ratio = 4.0     # typical ratio for feed-forward layer size

# Compute total FLOPS for the inference experiment
total_inference_flops = flops_for_experiment(
    num_steps=num_steps,
    batch_size=batch_size,
    seq_len=seq_len,
    hidden_dim=hidden_dim,
    num_layers=num_layers,
    num_heads=num_heads,
    ffn_ratio=ffn_ratio,
    r=r,
    training=True
)

print("=== Experiment FLOPS Calculation ===")
print(f"Number of optimizer steps: {num_steps}")
print(f"Batch size: {batch_size}")
print(f"Sequence length: {seq_len}")
print(f"Hidden dimension: {hidden_dim}")
print(f"Transformer blocks: {num_layers}")
print(f"Attention heads: {num_heads}")
print(f"FFN ratio: {ffn_ratio}")
print("------------------------------------")
print(f"Total FLOPS for this experiment: {np.log10(total_inference_flops)}")




=== Experiment FLOPS Calculation ===
Number of optimizer steps: 500
Batch size: 4
Sequence length: 512
Hidden dimension: 896
Transformer blocks: 24
Attention heads: 14
FFN ratio: 4.0
------------------------------------
Total FLOPS for this experiment: 15.381090216403893


In [10]:
10**17 / (10**12.682 *0.7 +10**12.682 *10 + 10**12.061 +10**12.871 *4)

1214.7513152658278

# LORA

## Section 3(a): Adapt the lora implementation, and train the 0.5B parameters Qwen model with default hyperparameters.

In [12]:
from src.lora_skeleton import apply_lora, load_data, train_lora

### load data

In [13]:
train_ids, val_ids, test_ids = load_data(tokenizer)

### load model

In [14]:
lr1_rank4_model,_ = load_qwen()
apply_lora(lr1_rank4_model, r=4)

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): LoRALinear(
            (original_linear): Linear(in_features=896, out_features=896, bias=True)
          )
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): LoRALinear(
            (original_linear): Linear(in_features=896, out_features=128, bias=True)
          )
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm(

### Train model


In [15]:
wandb.init(project="Qwen_3a_train_model", name="default_value_train_model", reinit=True)
losses_default_train=train_lora(lr1_rank4_model, train_ids, max_steps=500)

avg_loss,▁
avg_mse,▁
avg_loss,1.785
avg_mse,0.27025


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
Training:  10%|█         | 50/500 [01:07<09:03,  1.21s/it, loss=0.766]

Step 50: loss = 0.7663


Training:  20%|██        | 100/500 [02:07<07:36,  1.14s/it, loss=0.795]

...

Step 400: loss = 0.6226


Training:  90%|█████████ | 450/500 [09:03<01:01,  1.24s/it, loss=0.542]

Step 450: loss = 0.5419


Training: 100%|██████████| 500/500 [10:01<00:00,  1.20s/it, loss=0.685]


Step 500: loss = 0.6846


### Evalutate the model on validation dataset

In [20]:
wandb.init(project="Qwen_3a_train_model", name="default_value_evaluation_model", reinit=True)
val_loss_1_4, val_mse_1_4 = evaluation(lr1_rank4_model, tokenizer, tokenized_val,context_ratio=0.7)
print(f"Validation Loss: {val_loss_1_4:.4f} "
      f"Validation MSE (forecast): {val_mse_1_4:.4f}")



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/opt/anaconda3/envs/qwen/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/anaconda3/envs/qwen/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:634: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/opt/anaconda3/envs/qwen/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:651: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


Validation Loss: 1.6563 Validation MSE (forecast): 0.1248


### Reload the untrained QWen model, apply LoRA on the untrained model

In [21]:
untrained_model, _ = load_qwen()
apply_lora(untrained_model, r=4)
untrained_model.to(device)
wandb.init(project="Qwen_3a_train_model", name="LoRA_untrained_evaluation", reinit=True)

val_lora_untrained_loss, val_lora_untrained_mse = evaluation(untrained_model, tokenizer, tokenized_val, context_ratio=0.7)
print(f"Validation Loss: {val_lora_untrained_loss:.4f}")
print(f"Validation MSE (forecast): {val_lora_untrained_mse:.4f}")

avg_loss,▁
avg_mse,▁
avg_loss,1.65628
avg_mse,0.1248


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/opt/anaconda3/envs/qwen/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/anaconda3/envs/qwen/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:634: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/opt/anaconda3/envs/qwen/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:651: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


Validation Loss: 1.7850
Validation MSE (forecast): 0.2702


## 3(b): Make hyperparameter tuning and using metrics to select a best model.

### Implement a new method for evaluate the validation dataset by using dataloader. Previously we predicted many tokens for each of the 10 sequences, so we can check the performance for untrained/trained and LoRA/non-LoRA models. In this step, we do not explictly compare the performance. Instead, we use metrics, so we use the whole validation dataset but only predict 20-30 tokens for each of all the sequences.

In [22]:
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm 
from accelerate import Accelerator   
def evaluate_lora_mse(model, tokenizer, val_ids, context_ratio=0.5, batch_size=4, max_gen_tokens=30):
    """
    Evaluate a trained model on validation data using MSE.

    Args:
        model: The trained Qwen2.5 model (with LoRA applied).
        tokenizer: The tokenizer used for encoding/decoding.
        val_ids: A tensor of shape [num_sequences, seq_len].
        context_ratio: The percentage of tokens to use as context (e.g. 0.7).
        batch_size: Batch size for evaluation.

    Returns:
        avg_mse: Average Mean Squared Error over all sequences.
    """
    model.eval()
    val_dataset = TensorDataset(val_ids)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    accelerator = Accelerator()
    model, val_loader = accelerator.prepare(model, val_loader)


    mses = []
    losses = []

    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Evaluating"):
        # batch: (batch_size, seq_len)
            input_seqs = batch[0]

            for seq in input_seqs:
                context_ids, target_ids = split_context_target(seq, context_ratio)
                target_ids = target_ids[:max_gen_tokens]

                # 1. Cross-Entropy Loss over context+target
                full_seq = seq.unsqueeze(0)
                output = model(full_seq, labels=full_seq)
                losses.append(output.loss.item())

                # 2. Generation & MSE calculation
                input_ids = context_ids.unsqueeze(0)
                max_new_tokens = len(target_ids)
                generated = model.generate(
                    input_ids,
                    max_new_tokens=max_new_tokens,
                    do_sample=False
                )

                generated_ids = generated[0][len(context_ids):]
                pred_text = tokenizer.decode(generated_ids, skip_special_tokens=True)
                true_text = tokenizer.decode(target_ids, skip_special_tokens=True)

                pred_numbers = decode_tokens_to_numbers(pred_text)
                true_numbers = decode_tokens_to_numbers(true_text)

                if len(pred_numbers) == len(true_numbers) and len(pred_numbers) > 0:
                    mse = np.mean((np.array(pred_numbers) - np.array(true_numbers)) ** 2)
                    mses.append(mse)

    avg_ce_loss = np.mean(losses) if len(losses) > 0 else float("inf")
    avg_mse = np.mean(mses) if len(mses) > 0 else float("inf")

    print(f"\nEvaluation Results:")
    print(f"  Average Cross-Entropy Loss: {avg_ce_loss:.4f}")
    print(f"  Average MSE (forecast):     {avg_mse:.4f}")

    return avg_ce_loss, avg_mse

In [23]:
def evaluation(model, tokenizer, tokenized_data, context_ratio: float = 0.7):
    """
    Evaluates the model in a fully autoregressive manner using model.generate.
    
    For each sequence:
      1. Split the sequence into context and target using split_context_target.
      2. Use model.generate (with output_scores=True and return_dict_in_generate=True)
         to generate all target tokens at once.
      3. Extract the per-token logits (scores) for each generated token and compute the 
         cross-entropy loss against the ground truth token.
      4. Compute the average loss over the generated tokens and log the loss curve.
      5. Decode the generated tokens and the ground truth target tokens into numeric values,
         and compute the Mean Squared Error (MSE) for forecast evaluation.
    
    Args:
        model: The Qwen2.5-Instruct model.
        tokenizer: The corresponding tokenizer.
        tokenized_data (List[torch.Tensor]): List of 1D token ID tensors.
        context_ratio (float): Fraction of tokens used as context.
    
    Returns:
        Tuple[float, float]: The average cross-entropy loss and MSE over evaluated sequences.
    """
   
    num_eval = 10  # Evaluate first 10 sequences
    all_seq_losses = []
    all_seq_mses = []
    
    for i in range(num_eval):
        # Retrieve sequence and split into context and target
        seq = tokenized_data[i].to(device)
        context_ids, target_ids = split_context_target(seq, context_ratio)
        target_ids = target_ids[:100]
        input_ids = context_ids.unsqueeze(0)  # Shape: (1, context_length)
        
        # Generate tokens autoregressively using model.generate with scores output
        with torch.no_grad():
            gen_output = model.generate(
                input_ids,
                max_new_tokens=len(target_ids),
                do_sample=False,  # Greedy decoding
                output_scores=True,
                return_dict_in_generate=True
            )
        
        # gen_output.sequences contains context + generated tokens.
        generated_ids = gen_output.sequences[0]
        # gen_output.scores is a tuple of logits for each generated token, each with shape (batch_size, vocab_size)
        scores = gen_output.scores
        
        # Compute per-token loss using the returned scores and corresponding ground truth token
        token_losses = []
        for j, score in enumerate(scores):
            # Ground truth token for step j is target_ids[j]
            gt_token = target_ids[j].unsqueeze(0)  # Shape: (1,)
            loss_j = torch.nn.functional.cross_entropy(score, gt_token)
            token_losses.append(loss_j.item())
        avg_loss_seq = np.mean(token_losses)
        all_seq_losses.append(avg_loss_seq)
        
        # For forecast evaluation, compare generated tokens (excluding context) to ground truth target tokens
        generated_target_ids = generated_ids[len(context_ids):]
        pred_text = tokenizer.decode(generated_target_ids, skip_special_tokens=True)
        true_text = tokenizer.decode(target_ids, skip_special_tokens=True)
        pred_numbers = decode_tokens_to_numbers(pred_text)
        true_numbers = decode_tokens_to_numbers(true_text)
        mse = float("inf")
        if len(pred_numbers) == len(true_numbers) and len(pred_numbers) > 0:
            mse = np.mean((np.array(pred_numbers) - np.array(true_numbers)) ** 2)
        all_seq_mses.append(mse)
        
        # plot and log the loss curve
       
        plt.figure()
        plt.plot(token_losses, label="Token Loss")
        plt.title(f"Loss Curve for Sequence {i}")
        plt.xlabel("Prediction Step")
        plt.ylabel("Cross-Entropy Loss")
        plt.legend()
        wandb.log({f"loss_curve_seq_{i}": wandb.Image(plt)})
        plt.close()
        
        # plot and log forecast comparison (requires numeric reshaping, e.g., into (-1, 2))
        try:
            pred_array = np.array(pred_numbers).reshape(-1, 2)
            true_array = np.array(true_numbers).reshape(-1, 2)
            plt.figure()
            plt.plot(true_array[:, 0], label="True Prey")
            plt.plot(true_array[:, 1], label="True Predator", linestyle="--")
            plt.plot(pred_array[:, 0], label="Predicted Prey")
            plt.plot(pred_array[:, 1], label="Predicted Predator", linestyle="--")
            plt.title(f"Forecast Comparison for Sequence {i}", fontsize=15)
            plt.xlabel("Time Step", fontsize=15)
            plt.ylabel("Population", fontsize=15)   
            plt.legend(fontsize=12)
            wandb.log({f"forecast_seq_{i}": wandb.Image(plt)})
            plt.close()
        except Exception as e:
            print(f"Sequence {i}: Error in plotting forecast: {e}")
        
    
    avg_loss_overall = np.mean(all_seq_losses) if all_seq_losses else float("inf")
    avg_mse_overall = np.mean(all_seq_mses) if all_seq_mses else float("inf")
    wandb.log({
        "avg_loss": avg_loss_overall,
        "avg_mse": avg_mse_overall,
    })
    return avg_loss_overall, avg_mse_overall

In [24]:
wandb.init(project="Qwen_3a_train_model", name="default_value_evaluation_model", reinit=True)
val_loss_1_4_new, val_mse_1_4_new = evaluation(lr1_rank4_model, tokenizer, tokenized_val,context_ratio=0.7)
print(f"Validation Loss: {val_loss_1_4_new:.4f} "
      f"Validation MSE (forecast): {val_mse_1_4_new:.4f}")


avg_loss,▁
avg_mse,▁
avg_loss,1.785
avg_mse,0.27025


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/opt/anaconda3/envs/qwen/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/anaconda3/envs/qwen/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:634: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/opt/anaconda3/envs/qwen/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:651: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


Validation Loss: 1.6563 Validation MSE (forecast): 0.1248
